In [1]:
from collections import defaultdict
from datetime import datetime
from dateutil import tz
import os
os.chdir("/home/jovyan/work")

import arrow

from src.utils import send_request
from src._generate_msf_play_by_play_file import _get_play_type, UNMARSHALL, PARSED_GAMES

In [2]:
seasons = [
    "2015-2016-regular",
    "2016-playoff",
    "2016-2017-regular",
    "2017-playoff",
    "2017-2018-regular"
]

In [3]:
def unmarshall_season_game(game):
    date = get_datetime(game["schedule"]["homeTeam"]["abbreviation"], game["schedule"]["startTime"]).strftime("%Y%m%d")
    away = game["schedule"]["awayTeam"]["abbreviation"]
    home = game["schedule"]["homeTeam"]["abbreviation"]
    return "-".join([date, away, home])

In [4]:
def get_datetime(team_abbreviaion, dt):
    if team_abbreviaion in [
        "NYK", "BRO", "BOS", "PHI", 
        "WAS", "CHA", "ATL", "ORL", 
        "MIA", "TOR", "CLE", "DET", "IND"]:
        time_zone = tz.gettz("US/Eastern")
    elif team_abbreviaion in [
        "CHI", "MIN", "DAL", "SAS", 
        "OKL", "MEM", "HOU", "MIL", "NOP"]:
        time_zone = tz.gettz("US/Central")
    elif team_abbreviaion in ["DEN", "PHX", "UTA"]:
        time_zone = tz.gettz("US/Mountain")
    elif team_abbreviaion in [
        "POR", "GSW", "LAL", "LAC", "SAC"]:
        time_zone = tz.gettz("US/Pacific")
    else:
        print("Error with timezones")
    return datetime.fromtimestamp(arrow.get(dt).timestamp, time_zone)

In [5]:
games = []
for season in seasons:
    print("Processing: {season}".format(**locals()))
    fetched_season = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/{season}/games.json".format(**locals()))
    for game in fetched_season.get("games"):
        games.append({"season": season, "game": unmarshall_season_game(game), "id": game["schedule"]["id"]})

Processing: 2015-2016-regular
Processing: 2016-playoff
Processing: 2016-2017-regular
Processing: 2017-playoff
Processing: 2017-2018-regular


In [6]:
for g in games:
    #print("Processing {game} in {season}".format(**g))
    s = "https://api.mysportsfeeds.com/v2.0/pull/nba/{season}/games/{game}/playbyplay.json".format(**g)
    try:
        pbp = send_request(s)
        print(">> data for {game} : {id}\n".format(**g))
        for play in pbp.get("plays"):
            play_type = _get_play_type(play)
            UNMARSHALL[play_type](play)
    except ValueError:
        print("No data for {game} : {id}".format(**g))
        print(s)

>> data for 20151027-DET-ATL : 31176

>> data for 20151027-CLE-CHI : 31177

>> data for 20151027-NOP-GSW : 31178

>> data for 20151028-WAS-ORL : 31188

>> data for 20151028-PHI-BOS : 31179

>> data for 20151028-CHI-BRO : 31180

>> data for 20151028-UTA-DET : 31181

>> data for 20151028-CHA-MIA : 31185

>> data for 20151028-IND-TOR : 31192

>> data for 20151028-DEN-HOU : 31182

>> data for 20151028-CLE-MEM : 31184

>> data for 20151028-NYK-MIL : 31186

>> data for 20151028-SAS-OKL : 31187

>> data for 20151028-DAL-PHX : 31189

>> data for 20151028-NOP-POR : 31190

>> data for 20151028-LAC-SAC : 31191

>> data for 20151028-MIN-LAL : 31183

>> data for 20151029-MEM-IND : 31193

>> data for 20151029-ATL-NYK : 31195

>> data for 20151029-DAL-LAC : 31194

>> data for 20151030-MIA-CLE : 31198

>> data for 20151030-OKL-ORL : 31203

>> data for 20151030-UTA-PHI : 31204

>> data for 20151030-TOR-BOS : 31197

>> data for 20151030-CHI-DET : 31200

>> data for 20151030-CHA-ATL : 31196

>> data for 

>> data for 20151125-PHI-BOS : 31390

>> data for 20151125-MIA-DET : 31392

>> data for 20151125-CLE-TOR : 31401

>> data for 20151125-MEM-HOU : 31393

>> data for 20151125-SAC-MIL : 31395

>> data for 20151125-ATL-MIN : 31396

>> data for 20151125-BRO-OKL : 31397

>> data for 20151125-DAL-SAS : 31400

>> data for 20151125-NOP-PHX : 31399

>> data for 20151125-UTA-LAC : 31394

>> data for 20151127-CLE-CHA : 31403

>> data for 20151127-MIL-ORL : 31411

>> data for 20151127-WAS-BOS : 31402

>> data for 20151127-MIA-NYK : 31409

>> data for 20151127-PHI-HOU : 31405

>> data for 20151127-CHI-IND : 31406

>> data for 20151127-ATL-MEM : 31408

>> data for 20151127-DET-OKL : 31410

>> data for 20151127-SAS-DEN : 31404

>> data for 20151127-GSW-PHX : 31412

>> data for 20151127-MIN-SAC : 31413

>> data for 20151127-NOP-LAC : 31407

>> data for 20151128-TOR-WAS : 31420

>> data for 20151128-BRO-CLE : 31414

>> data for 20151128-DEN-DAL : 31415

>> data for 20151128-ATL-SAS : 31418

>> data for 

No data for 20151223-SAS-MIN : 31606
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151223-SAS-MIN/playbyplay.json
No data for 20151223-POR-NOP : 31607
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151223-POR-NOP/playbyplay.json
No data for 20151223-DEN-PHX : 31609
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151223-DEN-PHX/playbyplay.json
No data for 20151223-UTA-GSW : 31602
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151223-UTA-GSW/playbyplay.json
No data for 20151223-OKL-LAL : 31604
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151223-OKL-LAL/playbyplay.json
No data for 20151225-CHI-OKL : 31615
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151225-CHI-OKL/playbyplay.json
No data for 20151225-NOP-MIA : 32405
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20151225-NOP-MIA/playbyplay.json
No data for 20151225-CLE-GSW : 316

No data for 20160101-DAL-MIA : 31666
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160101-DAL-MIA/playbyplay.json
No data for 20160101-CHA-TOR : 31667
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160101-CHA-TOR/playbyplay.json
No data for 20160101-NYK-CHI : 31664
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160101-NYK-CHI/playbyplay.json
No data for 20160101-PHI-LAL : 31665
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160101-PHI-LAL/playbyplay.json
No data for 20160102-BRO-BOS : 31669
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160102-BRO-BOS/playbyplay.json
No data for 20160102-PHX-SAC : 31677
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160102-PHX-SAC/playbyplay.json
No data for 20160102-OKL-CHA : 31670
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160102-OKL-CHA/playbyplay.json
No data for 20160102-DET-IND : 316

No data for 20160109-CHI-ATL : 31724
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-CHI-ATL/playbyplay.json
No data for 20160109-WAS-ORL : 31727
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-WAS-ORL/playbyplay.json
No data for 20160109-BRO-DET : 31725
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-BRO-DET/playbyplay.json
No data for 20160109-TOR-PHI : 31728
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-TOR-PHI/playbyplay.json
No data for 20160109-MIA-UTA : 31730
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-MIA-UTA/playbyplay.json
No data for 20160109-GSW-SAC : 31729
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160109-GSW-SAC/playbyplay.json
No data for 20160110-NOP-LAC : 31733
https://api.mysportsfeeds.com/v2.0/pull/nba/2015-2016-regular/games/20160110-NOP-LAC/playbyplay.json
No data for 20160110-DAL-MIN : 317

...

>> data for 20180103-HOU-ORL : 42625

>> data for 20180103-DET-MIA : 42626

>> data for 20180103-MIN-BRO : 42627

>> data for 20180103-CLE-BOS : 42628

>> data for 20180103-IND-MIL : 42629

>> data for 20180103-TOR-CHI : 42630

>> data for 20180103-GSW-DAL : 42631

>> data for 20180103-PHX-DEN : 42632

>> data for 20180103-NOP-UTA : 42633

>> data for 20180103-OKL-LAL : 42634

>> data for 20180104-GSW-HOU : 42635

>> data for 20180104-OKL-LAC : 42636

>> data for 20180105-DET-PHI : 42638

>> data for 20180105-MIN-BOS : 42639

>> data for 20180105-NYK-MIA : 42637

>> data for 20180105-TOR-MIL : 42640

>> data for 20180105-CHI-DAL : 42641

>> data for 20180105-PHX-SAS : 42642

>> data for 20180105-UTA-DEN : 42643

>> data for 20180105-WAS-MEM : 42644

>> data for 20180105-ATL-POR : 42645

>> data for 20180105-CHA-LAL : 42646

>> data for 20180106-GSW-LAC : 42647

>> data for 20180106-BOS-BRO : 42648

>> data for 20180106-HOU-DET : 42649

>> data for 20180106-CLE-ORL : 42650

>> data for 

>> data for 20180202-UTA-PHX : 42841

>> data for 20180202-NOP-OKL : 42842

>> data for 20180202-GSW-SAC : 42843

>> data for 20180203-CHI-LAC : 42844

>> data for 20180203-MIA-DET : 42845

>> data for 20180203-WAS-ORL : 42846

>> data for 20180203-PHI-IND : 42847

>> data for 20180203-HOU-CLE : 42848

>> data for 20180203-UTA-SAS : 42849

>> data for 20180203-GSW-DEN : 42850

>> data for 20180203-NOP-MIN : 42851

>> data for 20180203-DAL-SAC : 42852

>> data for 20180204-POR-BOS : 42853

>> data for 20180204-MIL-BRO : 42854

>> data for 20180204-MEM-TOR : 42855

>> data for 20180204-ATL-NYK : 42856

>> data for 20180204-LAL-OKL : 42857

>> data for 20180204-CHA-PHX : 42858

>> data for 20180205-WAS-IND : 42859

>> data for 20180205-POR-DET : 42860

>> data for 20180205-ORL-MIA : 42861

>> data for 20180205-UTA-NOP : 42862

>> data for 20180205-CHA-DEN : 42863

>> data for 20180205-CHI-SAC : 42864

>> data for 20180205-DAL-LAC : 42865

>> data for 20180206-CLE-ORL : 42867

>> data for 

>> data for 20180310-SAS-OKL : 43058

>> data for 20180310-MEM-DAL : 43059

>> data for 20180310-ORL-LAC : 43060

>> data for 20180311-TOR-NYK : 43291

>> data for 20180311-CHI-ATL : 43293

>> data for 20180311-GSW-MIN : 43294

>> data for 20180311-UTA-NOP : 43295

>> data for 20180311-SAC-DEN : 43296

>> data for 20180311-HOU-DAL : 43297

>> data for 20180311-PHI-BRO : 43292

>> data for 20180311-IND-BOS : 43298

>> data for 20180311-CLE-LAL : 43299

>> data for 20180312-MIL-MEM : 43061

>> data for 20180312-SAS-HOU : 43062

>> data for 20180312-SAC-OKL : 43063

>> data for 20180312-MIA-POR : 43064

>> data for 20180313-IND-PHI : 43065

>> data for 20180313-MIN-WAS : 43066

>> data for 20180313-DAL-NYK : 43067

>> data for 20180313-OKL-ATL : 43068

>> data for 20180313-TOR-BRO : 43069

>> data for 20180313-LAC-CHI : 43070

>> data for 20180313-CHA-NOP : 43071

>> data for 20180313-ORL-SAS : 43072

>> data for 20180313-DET-UTA : 43073

>> data for 20180313-CLE-PHX : 43074

>> data for 

No data for 20180403-WAS-HOU : 43222
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-WAS-HOU/playbyplay.json
No data for 20180403-GSW-OKL : 43223
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-GSW-OKL/playbyplay.json
No data for 20180403-POR-DAL : 43224
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-POR-DAL/playbyplay.json
No data for 20180403-IND-DEN : 43225
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-IND-DEN/playbyplay.json
No data for 20180403-LAL-UTA : 43226
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-LAL-UTA/playbyplay.json
No data for 20180403-SAC-PHX : 43227
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-SAC-PHX/playbyplay.json
No data for 20180403-SAS-LAC : 43228
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180403-SAS-LAC/playbyplay.json
No data for 20180404-DAL-ORL : 432

No data for 20180411-WAS-ORL : 43282
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-WAS-ORL/playbyplay.json
No data for 20180411-DET-CHI : 43283
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-DET-CHI/playbyplay.json
No data for 20180411-MEM-OKL : 43284
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-MEM-OKL/playbyplay.json
No data for 20180411-BRO-BOS : 43285
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-BRO-BOS/playbyplay.json
No data for 20180411-DEN-MIN : 43286
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-DEN-MIN/playbyplay.json
No data for 20180411-NYK-CLE : 43287
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-NYK-CLE/playbyplay.json
No data for 20180411-LAL-LAC : 43288
https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games/20180411-LAL-LAC/playbyplay.json
No data for 20180411-HOU-SAC : 432

In [7]:
plays = 0
for k in PARSED_GAMES.keys():
    plays += len(PARSED_GAMES.get(k))

plays

1348554